In [ ]:
#Notebook Goals
#The goal of this notebook is the use the Five C's of credit framework (Character, Capacity, Cash Flow, Collateral, and Condition) to select the proxy features from the Kaggle Lending Club Dataset to predict loan default probability.

#Using the predicted loan default probability, the notebook will the generate expected loss on loans within the portfolio.

#Expected Loss (EL) on a given is calculuated using the loan amount multiplied by a loan's probability of default (PD) mulitplied by the the loan's loss given default (LGD). Loss Given Default is the amount of the loan that is deemed unrecoverable and is represented as a number between 0 (no loss) and 1 (100% of the loan is a loss)

#The equation for EL is given as:

# EL = PD x LGD x Loan Amount

#After selected features using the 5 C's of Credit Framework and adjusting for the skewedness in the data (there is a greater proportion of loans in the portfolio that have NOT defaulted versus loans that HAVE defaulted) we will model proabilities of default using classification models.

#With proabilties of default we will then model Expected Loss using an assumed LGD of 1 (a LGD of 1 is a conservative assumption given the unsecured nature of the loans)

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [6]:
raw_data = pd.read_csv("loan.csv", low_memory=False, nrows=300000)

In [7]:
raw_data.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
raw_data.shape

(300000, 145)

In [10]:
preprocess_df = raw_data[['emp_length', 'loan_status', 'home_ownership', 'issue_d',
                          'earliest_cr_line', 'purpose', 'term', 'annual_inc', 'dti',
                        'loan_amnt', 'int_rate', 'pub_rec_bankruptcies']].copy()

In [11]:
preprocess_df.head(5)

,emp_length,loan_status,home_ownership,issue_d,earliest_cr_line,purpose,term,annual_inc,dti,loan_amnt,int_rate,pub_rec_bankruptcies
0,10+ years,Current,RENT,Dec-2018,Apr-2001,debt_consolidation,36 months,55000.0,18.24,2500,13.56,1
1,10+ years,Current,MORTGAGE,Dec-2018,Jun-1987,debt_consolidation,60 months,90000.0,26.52,30000,18.94,1
2,6 years,Current,MORTGAGE,Dec-2018,Apr-2011,debt_consolidation,36 months,59280.0,10.51,5000,17.97,0
3,10+ years,Current,MORTGAGE,Dec-2018,Feb-2006,debt_consolidation,36 months,92000.0,16.74,4000,18.94,0
4,10+ years,Current,MORTGAGE,Dec-2018,Dec-2000,debt_consolidation,60 months,57250.0,26.35,30000,16.14,0


In [59]:
preprocess_df = raw_data[['emp_length', 'loan_status', 'home_ownership', 'issue_d',
                          'earliest_cr_line', 'purpose', 'term', 'annual_inc', 'dti',
                          'loan_amnt', 'int_rate', 'pub_rec_bankruptcies',]].copy()

# Create a list of columns that are NOT numeric values
not_numeric_cols = ['emp_length', 'loan_status', 'home_ownership', 'issue_d',
                    'earliest_cr_line', 'purpose', 'term']

# Create list of columns that ARE numeric values and print
numeric_cols = [col for col in preprocess_df.columns if col not in not_numeric_cols]
print(numeric_cols)

# Convert numeric cols into numeric data types
preprocess_df[numeric_cols] = preprocess_df[numeric_cols].apply(pd.to_numeric)

['annual_inc', 'dti', 'loan_amnt', 'int_rate', 'pub_rec_bankruptcies']


In [60]:
# Create list of datetime columns
datetime_cols = ['earliest_cr_line', 'issue_d']

# Convert to datetime
preprocess_df[datetime_cols] = preprocess_df[datetime_cols].apply(pd.to_datetime)

In [61]:
# Create list of datetime columns
datetime_cols = ['earliest_cr_line', 'issue_d']

# Convert to datetime
preprocess_df[datetime_cols] = preprocess_df[datetime_cols].apply(pd.to_datetime)

In [62]:
### Missing data and observation

In [63]:
preprocess_df.isnull().sum()

emp_length              25554
loan_status                 0
home_ownership              0
issue_d                     0
earliest_cr_line            0
purpose                     0
term                        0
annual_inc                  0
dti                       654
loan_amnt                   0
int_rate                    0
pub_rec_bankruptcies        0
dtype: int64

In [64]:
character_df = preprocess_df[['pub_rec_bankruptcies', 'earliest_cr_line', 'issue_d']].copy()

In [65]:
print(character_df.head())
print(character_df.describe())
print(character_df.dtypes)

   pub_rec_bankruptcies earliest_cr_line    issue_d
0                     1       2001-04-01 2018-12-01
1                     1       1987-06-01 2018-12-01
2                     0       2011-04-01 2018-12-01
3                     0       2006-02-01 2018-12-01
4                     0       2000-12-01 2018-12-01
       pub_rec_bankruptcies
count         300000.000000
mean               0.123357
std                0.334834
min                0.000000
25%                0.000000
50%                0.000000
75%                0.000000
max                6.000000
pub_rec_bankruptcies             int64
earliest_cr_line        datetime64[ns]
issue_d                 datetime64[ns]
dtype: object


In [66]:
# fill the missing value for earliest_cr_line with most frequently occuring
character_df['earliest_cr_line'].fillna(character_df['earliest_cr_line'].value_counts().index[0], inplace=True)

In [67]:
# count months between now and 'earliest_cr_line'
character_df['credit_hist_in_months'] = ((character_df['issue_d'] - character_df['earliest_cr_line'])/np.timedelta64(1, 'M'))
character_df['credit_hist_in_months'] = character_df['credit_hist_in_months'].astype(int)

character_df.head()

,pub_rec_bankruptcies,earliest_cr_line,issue_d,credit_hist_in_months
0,1,2001-04-01,2018-12-01,212
1,1,1987-06-01,2018-12-01,378
2,0,2011-04-01,2018-12-01,92
3,0,2006-02-01,2018-12-01,153
4,0,2000-12-01,2018-12-01,215


In [68]:
# Create a new binary feature of whether or not there is a bankruptcy on file in customers credit history
character_df['cb_person_bk_on_file_Y'] = character_df['pub_rec_bankruptcies'].apply(lambda x: 1 if x >= 1 else 0)
character_df.head()

,pub_rec_bankruptcies,earliest_cr_line,issue_d,credit_hist_in_months,cb_person_bk_on_file_Y
0,1,2001-04-01,2018-12-01,212,1
1,1,1987-06-01,2018-12-01,378,1
2,0,2011-04-01,2018-12-01,92,0
3,0,2006-02-01,2018-12-01,153,0
4,0,2000-12-01,2018-12-01,215,0


In [69]:
# drop the old features from the character_df
#character_df.drop(['pub_rec_bankruptcies', 'earliest_cr_line', 'issue_d'], axis=1, inplace=True)
#character_df.head()

In [72]:
# Create a new binary feature of whether or not there is a bankruptcy on file in customers credit history
character_df['cb_person_bk_on_file_Y'] = character_df['pub_rec_bankruptcies'].apply(lambda x: 1 if x >= 1 else 0)
character_df.head()

,pub_rec_bankruptcies,earliest_cr_line,issue_d,credit_hist_in_months,cb_person_bk_on_file_Y
0,1,2001-04-01,2018-12-01,212,1
1,1,1987-06-01,2018-12-01,378,1
2,0,2011-04-01,2018-12-01,92,0
3,0,2006-02-01,2018-12-01,153,0
4,0,2000-12-01,2018-12-01,215,0


In [73]:
# fill the missing value for earliest_cr_line with most frequently occuring
character_df['earliest_cr_line'].fillna(character_df['earliest_cr_line'].value_counts().index[0], inplace=True)

In [74]:
# count months between now and 'earliest_cr_line'
character_df['credit_hist_in_months'] = ((character_df['issue_d'] - character_df['earliest_cr_line'])/np.timedelta64(1, 'M'))
character_df['credit_hist_in_months'] = character_df['credit_hist_in_months'].astype(int)

character_df.head()

,pub_rec_bankruptcies,earliest_cr_line,issue_d,credit_hist_in_months,cb_person_bk_on_file_Y
0,1,2001-04-01,2018-12-01,212,1
1,1,1987-06-01,2018-12-01,378,1
2,0,2011-04-01,2018-12-01,92,0
3,0,2006-02-01,2018-12-01,153,0
4,0,2000-12-01,2018-12-01,215,0


In [75]:
# Create a new binary feature of whether or not there is a bankruptcy on file in customers credit history
character_df['cb_person_bk_on_file_Y'] = character_df['pub_rec_bankruptcies'].apply(lambda x: 1 if x >= 1 else 0)
character_df.head()

,pub_rec_bankruptcies,earliest_cr_line,issue_d,credit_hist_in_months,cb_person_bk_on_file_Y
0,1,2001-04-01,2018-12-01,212,1
1,1,1987-06-01,2018-12-01,378,1
2,0,2011-04-01,2018-12-01,92,0
3,0,2006-02-01,2018-12-01,153,0
4,0,2000-12-01,2018-12-01,215,0


In [76]:
# drop the old features from the character_df
character_df.drop(['pub_rec_bankruptcies', 'earliest_cr_line', 'issue_d'], axis=1, inplace=True)
character_df.head()

,credit_hist_in_months,cb_person_bk_on_file_Y
0,212,1
1,378,1
2,92,0
3,153,0
4,215,0


In [ ]:
#Capacity/Cash flow: annual_inc & dti
#What it is: Your ability to repay the loan. The ability to generate cash to repay all oblgiations, when due. Assessing this is the primary goal of credit analysis

#Why it matters: Lenders want to be assured that your business generates enough cash flow to repay the loan in full.

In [77]:
capacity_df = preprocess_df[['annual_inc', 'dti']].copy()
capacity_df.head()

,annual_inc,dti
0,55000.0,18.24
1,90000.0,26.52
2,59280.0,10.51
3,92000.0,16.74
4,57250.0,26.35


In [81]:
# fill missing values for annual income with the mean
capacity_df['annual_inc'] = capacity_df['annual_inc'].fillna(capacity_df['annual_inc'].mean())

# fill missing values for dti with the mean
capacity_df['dti'] = capacity_df['dti'].fillna(capacity_df['dti'].mean())

In [82]:

# describe the capacity/cash flow proxy features
print(capacity_df[['annual_inc', 'dti']].describe())

         annual_inc            dti
count  3.000000e+05  300000.000000
mean   8.109447e+04      19.751158
std    9.546078e+04      20.081154
min    0.000000e+00       0.000000
25%    4.671975e+04      11.580000
50%    6.700000e+04      17.820000
75%    9.700000e+04      25.050000
max    9.757200e+06     999.000000


In [ ]:
#Capital¶
#What it is: The amount of money invested by the business owner or management team. It is required during periods of weak cash flow generation for an obligor to sustain itself and meeet obligations. If there isn't sufficient cash flow to meet obligations, then assets may need to be sold to meet obligations.

#Why it matters: Banks are more willing to lend to owners who have invested some of their own money into the venture. It shows you have some “skin in the game.”

#How it’s assessed: From the amount of money the borrower or management team has invested in the business.

In [ ]:
#Conditions : loan_amnt, int_rate, term¶
#What it is: The condition of a business/individual — whether it is growing or faltering — as well as what you’ll use the funds for. Refers to the macro-economic and competitive environment which will impact an obligor's future performance and its ability to generate cash.

#Why it matters: To ensure that loans are repaid, banks want to lend to businesses operating under favorable conditions. They aim to identify risks and protect themselves accordingly.

#How it’s assessed: From a review of the competitive landscape, supplier and customer relationships, and macroeconomic and industry-specific issues.

In [83]:

conditions_df = preprocess_df[['loan_amnt', 'int_rate', 'term']].copy()

In [84]:
conditions_df.term.value_counts()

 36 months    208021
 60 months     91979
Name: term, dtype: int64

In [85]:
# Convert values of term to 0, 1 where 0 = 36 months and 1 = 60 months
conditions_df['term'] = conditions_df['term'].replace({' 36 months': '0',
                                                       ' 60 months': '1'})

# convert term into an integer data type 
conditions_df['term'] = conditions_df['term'].astype(int)

# Rename term column
conditions_df = conditions_df.rename(columns={'term': 'term_60'})

print(conditions_df.term_60.value_counts())
conditions_df.head()

0    208021
1     91979
Name: term_60, dtype: int64


,loan_amnt,int_rate,term_60
0,2500,13.56,0
1,30000,18.94,1
2,5000,17.97,0
3,4000,18.94,0
4,30000,16.14,1
